In [1]:
import os
import sys
sys.path.insert(0,'../')
from libs.ssl_dataloader import *
from libs.ssl_model import *
from libs.ssl_utils import *
from libs.eeg_utils import *
from braindecode.preprocessing import (
    preprocess, Preprocessor, create_fixed_length_windows)
from braindecode.datasets import BaseDataset, BaseConcatDataset
%load_ext autoreload
%autoreload 2
import torch
from torch import nn, optim
from torch.utils.data import DataLoader

In [ ]:
datasets = []
hbn_datasets = ['ds005514','ds005512','ds005511','ds005510','ds005509','ds005508','ds005507','ds005506','ds005505']

In [15]:
ds1 = HBNDataset('ds005505', tasks=['RestingState'], subjects=['NDARRD326KB9', 'NDARZD415ZZ1', 'NDARXH597ML1'])
ds2 = HBNDataset('ds005510', tasks=['RestingState'], subjects=['NDARAE877NER', 'NDARYW984FLT', 'NDARFX710UZA'])

Loading /mnt/nemar/openneuro/ds005505/sub-NDARRD326KB9/eeg/sub-NDARRD326KB9_task-RestingState_eeg.set
Loading /mnt/nemar/openneuro/ds005505/sub-NDARZD415ZZ1/eeg/sub-NDARZD415ZZ1_task-RestingState_eeg.set
Loading /mnt/nemar/openneuro/ds005505/sub-NDARXH597ML1/eeg/sub-NDARXH597ML1_task-RestingState_eeg.set
Loading /mnt/nemar/openneuro/ds005510/sub-NDARAE877NER/eeg/sub-NDARAE877NER_task-RestingState_eeg.set
Loading /mnt/nemar/openneuro/ds005510/sub-NDARYW984FLT/eeg/sub-NDARYW984FLT_task-RestingState_eeg.set
Loading /mnt/nemar/openneuro/ds005510/sub-NDARFX710UZA/eeg/sub-NDARFX710UZA_task-RestingState_eeg.set


In [16]:
all_ds = BaseConcatDataset([ds1, ds2])

In [17]:
window_len_s = 2
fs = all_ds.datasets[0].raw.info['sfreq']
window_len_samples = int(fs * window_len_s)
window_stride_samples = int(fs * 4)
# window_stride_samples = int(fs * window_len_s)
windows_ds = create_fixed_length_windows(
    all_ds, start_offset_samples=0, stop_offset_samples=None,
    window_size_samples=window_len_samples,
    window_stride_samples=window_stride_samples, drop_last_window=True,
    preload=False)

In [18]:
dataloader = DataLoader(
        windows_ds, batch_size=12, shuffle=False, pin_memory=False,
        num_workers=3, worker_init_fn=None)
for X, y, _ in dataloader:
    print(X)
    print(X.shape)
    break

tensor([[[-6.6219e-03, -6.6984e-03, -6.7410e-03,  ..., -6.7668e-03,
          -6.6746e-03, -6.6051e-03],
         [-6.3035e-03, -6.3509e-03, -6.4162e-03,  ..., -6.4719e-03,
          -6.3800e-03, -6.2900e-03],
         [-8.3630e-03, -8.4100e-03, -8.4701e-03,  ..., -8.5641e-03,
          -8.4676e-03, -8.3817e-03],
         ...,
         [ 1.8477e-04,  1.5949e-04,  9.7502e-05,  ..., -2.1671e-04,
          -1.3200e-04, -5.3741e-05],
         [-5.6502e-03, -5.7106e-03, -5.7451e-03,  ..., -5.9163e-03,
          -5.8231e-03, -5.7548e-03],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[-6.7643e-03, -6.8234e-03, -6.8756e-03,  ..., -6.8925e-03,
          -6.8053e-03, -6.7479e-03],
         [-6.4459e-03, -6.5046e-03, -6.5534e-03,  ..., -6.6023e-03,
          -6.5085e-03, -6.4453e-03],
         [-8.5088e-03, -8.5684e-03, -8.6178e-03,  ..., -8.6686e-03,
          -8.5737e-03, -8.5115e-03],
         ...,
         [-4.1474e-04, -4